### Trajectory Clustering Approaches with AIS Shipping Data

In [1]:
#import packages

import os
import glob
import pandas as pd
import numpy as np
import geopy.distance
import urllib
import zipfile
import scipy.io
import math

from dipy.segment.metric import Metric
from dipy.segment.metric import ResampleFeature
from dipy.segment.clustering import QuickBundles
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import directed_hausdorff

#### Data Loading and Pre-processing

In [2]:
#set working directory to folder with trajectory files
os.chdir(os.getcwd()+"/tracks")

In [3]:
#set display options
pd.set_option('display.max_rows', 3000)

### Set up Dataframe for EDA

To get a sense of it, would be useful to figure out the overall timeframe (earliest/latest observation overall), lat/long coverage (whole world or other), overall distribution of time intervals, and average speed between consecutive measurements, like we were doing for other datasets (I can share the plots and/or the code if you wish).

Figuring out the intervals during which each boat does not move (e.g. distance between each pair of measurements within a certain threshold or avg. speed between each pair of consecutive measurements within the interval not exceeding some threshold like 0.5mph; as some minimal movement can be detected just based on the measurement inaccuracies) would be useful. To implement that we can just traverse the trajectory and start accumulating “stationarity” interval each time we see a pair of consecutive measurements falls under speed threshold and keep accumulating them until the speed does not return to values above the threshold. If the stationarity period was long enough and satisfies some overall movement threshold we can then cut here and start a new “trip”. This way we split the trajectory into trips (segments of actual movement), but keep them grouped per vessel id. 

In [ ]:
#initialize an empty dataframe with the columns from the NPZ files
trajectories = pd.DataFrame(columns=['mmsi','timestamp','distance_from_shore','distance_from_port',
                                     'speed','course','lat','lon'])

counts_tracker = 0 #initialize counter for tracking

#load npz files from directory
for file in glob.glob('*.npz'):
    try:
        data = np.load(file) #load file
        lst = data.files
        for item in lst:
            new_traj = pd.DataFrame(data[item])
        trajectories = pd.concat([trajectories,new_traj])
        counts_tracker += 1
        print(counts_tracker) #track how many files have been loaded
    except:
        pass

#df clean up
trajectories.drop_duplicates(inplace=True) #drop duplicate entries
trajectories.timestamp  = pd.to_datetime(trajectories.timestamp,unit='s') #convert to time stamp
trajectories.reset_index(inplace=True) #reset
trajectories['mmsi'] = trajectories['mmsi'].astype('int64') #set trajectory values to raw numbers

In [ ]:
#export to csv for future use
trajectories.to_csv('trajectories.csv')

### Generate some basic stats

To get a sense of it, would be useful to figure out the overall timeframe (earliest/latest observation overall), lat/long coverage (whole world or other), overall distribution of time intervals, and average speed between consecutive measurements, like we were doing for other datasets (I can share the plots and/or the code if you wish).

In [ ]:
#number of unique boats
len(trajectories['mmsi'].unique())

In [ ]:
#min and max of time range on data set
print("Dates range from {} to {}".format(min(trajectories.timestamp),max(trajectories.timestamp)))

In [ ]:
#lat lon coverage
print("Latitude ranges from {} to {}".format(min(trajectories.lat),max(trajectories.lat)))
print("Longitude ranges from {} to {}".format(min(trajectories.lon),max(trajectories.lon)))

In [ ]:
#average number of 

In [ ]:
#shortest vs. longest trajectories

In [ ]:
#overall distribution of time intervals


In [ ]:
#average speed between consecutive measurements


### Division of trajectories into trips

In [ ]:
#take sample of trajectories for testing modeling
trajectories_test = trajectories[trajectories['mmsi']==['','','']]

### Set up data as arrays for modeling

In [5]:
boat_list = list(trajectories.mmsi.unique()) #get a list of unique boat ids
all_boat_locs = [] #initialize a list 

for i in boat_list:
    boat_loc = []
    boat = trajectories[trajectories.mmsi == i]
    for index, row in boat.iterrows():
        traj = [row['lat'],row['lon']]
        traj = np.array(traj)
        boat_loc.append(traj)
    boat_loc = np.array(boat_loc)
    all_boat_locs.append(boat_loc)
    
all_boat_locs = np.array(all_boat_locs) #convert to array

In [6]:
len(all_boat_locs)

95

#### Quick Bundles Approach Using DiPy

In [ ]:
#define class to calculate geographic distance
class GPSdistance(Metric):
    '''computes the average GPS distance between two trajectories'''
    def __init__(self):
        super(GPSdistance, self) \
        .__init__(feature=ResampleFeature(nb_points=
    min(trajectories.groupby(['mmsi'])['speed'].count())))
    
    def are_compatible(self, shape1, shape2):
        return len(shape1) == len(shape2)
    
    def dist(self, v1, v2):
        x = [geopy.distance.vincenty(
            [p[0][0],p[0][1], p[1][0],p[1][1]]).km for p in list(zip(v1,v2))]
        currD = np.mean(x)
        return currD

In [ ]:
#run quick bundles clustering
metric = GPSdistance() #need to get this to work so that it's using geographic distance
qb = QuickBundles(threshold=1)
clusters = qb.cluster(all_boat_locs)

In [ ]:
print("Nb. clusters:", len(clusters))
print("Cluster sizes:", list(map(len, clusters)))
print("Small clusters:", clusters < 10)
print("Streamlines indices of the first cluster:\n", clusters[0].indices)
print("Centroid of the last cluster:\n", clusters[-1].centroid)

In [ ]:
clusters[0]

### DBSCAN Approach